In [6]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import os
from datetime import datetime

def clean_district_name(name):
    """Clean district names by removing common prefixes/suffixes and standardizing format"""
    if not isinstance(name, str):
        return str(name)
    
    # Convert to lowercase for comparison
    name = name.lower()
    
    # Remove common prefixes
    prefixes = ['dr.', 'dr ', 'shri ', 'saint ', 'st.', 'st ', 'district']
    for prefix in prefixes:
        if name.startswith(prefix):
            name = name[len(prefix):].strip()
    
    # Remove special characters and extra spaces
    name = ' '.join(name.split())
    
    return name

def are_districts_similar(dist1, dist2, threshold=80):
    """Check if two district names are similar using multiple fuzzy matching approaches"""
    dist1 = clean_district_name(dist1)
    dist2 = clean_district_name(dist2)
    
    # Direct match after cleaning
    if dist1 == dist2:
        return True
    
    # Ratio match
    ratio = fuzz.ratio(dist1, dist2)
    if ratio >= threshold:
        return True
    
    # Partial ratio match (for substring matches)
    partial_ratio = fuzz.partial_ratio(dist1, dist2)
    if partial_ratio >= threshold:
        return True
    
    # Token sort ratio (for word order differences)
    token_sort_ratio = fuzz.token_sort_ratio(dist1, dist2)
    if token_sort_ratio >= threshold:
        return True
    
    # Token set ratio (for handling extra/missing words)
    token_set_ratio = fuzz.token_set_ratio(dist1, dist2)
    if token_set_ratio >= threshold:
        return True
    
    return False

def find_missing_districts(file_path, sheet1_name='Sheet1', sheet2_name='Sheet2'):
    """Find districts present in sheet1 but not in sheet2, accounting for fuzzy matching"""
    
    # Read both sheets
    df1 = pd.read_excel(file_path, sheet_name=sheet1_name)
    df2 = pd.read_excel(file_path, sheet_name=sheet2_name)
    
    # Initialize list to store missing districts
    missing_districts = []
    
    # Iterate through sheet1 districts
    for idx, row1 in df1.iterrows():
        state1 = row1['State']
        district1 = row1['District']
        
        # Get all districts from sheet2 for the same state
        state_districts_sheet2 = df2[df2['State'] == state1]['District'].tolist()
        
        # Check if district exists in sheet2 (using fuzzy matching)
        found_match = False
        for district2 in state_districts_sheet2:
            if are_districts_similar(district1, district2):
                found_match = True
                break
        
        # If no match found, add to missing districts
        if not found_match:
            missing_districts.append({
                'State': state1,
                'District': district1
            })
    
    # Create DataFrame from missing districts
    missing_df = pd.DataFrame(missing_districts)
    
    # Save results to desktop
    desktop_path = os.path.join(os.path.expanduser('~'), 'Desktop')
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_file = os.path.join(desktop_path, f'missing_districts_{timestamp}.xlsx')
    
    missing_df.to_excel(output_file, index=False)
    
    print(f"Found {len(missing_districts)} missing districts")
    print(f"Results saved to: {output_file}")
    
    return missing_df

if __name__ == "__main__":
    # Replace with your Excel file path
    excel_file = input("Enter the path to your Excel file: ")
    
    # Get sheet names from user (optional)
    sheet1 = input("Enter name of first sheet (default: Sheet1): ") or 'Sheet1'
    sheet2 = input("Enter name of second sheet (default: Sheet2): ") or 'Sheet2'
    
    # Run the comparison
    missing_districts = find_missing_districts(excel_file, sheet1, sheet2)
    
    # Display first few missing districts
    print("\nFirst few missing districts:")
    print(missing_districts.head())

KeyError: 'State'

In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Load the Excel file
file_path = r"C:\Users\ASUS\Downloads\Kill me.xlsx"  # Replace with your file path

# Read the sheets into DataFrames
table1 = pd.read_excel(file_path, sheet_name='RTO')
table2 = pd.read_excel(file_path, sheet_name='Dealer Master')

# Normalize the district names
table1['District'] = table1['District'].str.lower().str.strip()
table2['District'] = table2['District'].str.lower().str.strip()

# Function to find the best match
def find_best_match(district, choices, scorer, score_cutoff=80):
    best_match = process.extractOne(district, choices, scorer=scorer)
    if best_match and best_match[1] >= score_cutoff:
        return best_match[0]
    return None

# Find missing districts
missing_districts = []
for state in table2['State'].unique():
    state_districts_t1 = table1[table1['State'] == state]['District'].tolist()
    state_districts_t2 = table2[table2['State'] == state]['District'].tolist()

    for district in state_districts_t2:
        best_match = find_best_match(district, state_districts_t1, fuzz.token_sort_ratio)
        if not best_match:
            missing_districts.append((state, district))

# Output the missing districts
for state, district in missing_districts:
    print(f"State: {state}, District: {district}")


State: Andaman & Nicobar Island, District: 
State: Andhra Pradesh, District: yanam
State: Andhra Pradesh, District: kadapa
State: Andhra Pradesh, District: sri balaji (thirupati)
State: Andhra Pradesh, District: konaseema
State: Andhra Pradesh, District: n t rama rao
State: Andhra Pradesh, District: manyam
State: Assam, District: bajali
State: Assam, District: kamrup metropolitan
State: Assam, District: south salmara-mankachar
State: Bihar, District: kaimur
State: Bihar, District: east champaran
State: Bihar, District: west champaran
State: Bihar, District: saran
State: Chandigarh, District: chandigarh
State: Chhattisgarh, District: kanker
State: Chhattisgarh, District: mohla manpur
State: Chhattisgarh, District: khairagarh
State: Chhattisgarh, District: manendragarh
State: Chhattisgarh, District: sakti
State: Chhattisgarh, District: sarangarh bilaigarh
State: Delhi, District: new delhi
State: Delhi, District: north east delhi
State: Delhi, District: shahdara
State: Haryana, District: 

In [4]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Load the Excel file
file_path = r"C:\Users\ASUS\Downloads\Kill me.xlsx"  # Replace with your file path

# Read the sheets into DataFrames
table1 = pd.read_excel(file_path, sheet_name='RTO')
table2 = pd.read_excel(file_path, sheet_name='Dealer Master')

# Debugging: Print column names to verify
print("Columns in table1:", table1.columns)
print("Columns in table2:", table2.columns)

# Normalize the district names
table1['District'] = table1['District'].str.lower().str.strip()
table2['District'] = table2['District'].str.lower().str.strip()

# Function to find the best match
def find_best_match(district, choices, scorer, score_cutoff=80):
    best_match = process.extractOne(district, choices, scorer=scorer)
    if best_match and best_match[1] >= score_cutoff:
        return best_match[0]
    return None

# Find missing districts
missing_districts = []
for state in table2['State'].unique():
    state_districts_t1 = table1[table1['State'] == state]['District'].tolist()
    state_districts_t2 = table2[table2['State'] == state]['District'].tolist()

    for district in state_districts_t2:
        best_match = find_best_match(district, state_districts_t1, fuzz.token_sort_ratio)
        if not best_match:
            missing_districts.append((state, district))

# Create a DataFrame from the missing districts
missing_df = pd.DataFrame(missing_districts, columns=['State', 'District'])

# Save the DataFrame to a new Excel file
output_file_path = 'missing_districts.xlsx'  # Specify the output file path
missing_df.to_excel(output_file_path, index=False)

print(f"Missing districts have been saved to {output_file_path}")


Columns in table1: Index(['State', 'District'], dtype='object')
Columns in table2: Index(['State', 'District'], dtype='object')
Missing districts have been saved to missing_districts.xlsx
